# Aula 11 - Análise de Séries Temporais: Atividades Guiadas

**Programação para Ciência de Dados**

**Data:** 11 de Novembro de 2025

---

## Objetivos da Aula

Ao final desta aula, você será capaz de:

**Conhecimento:**
- Entender componentes de séries temporais (tendência, sazonalidade, resíduo)
- Conhecer desafios específicos de dados temporais
- Interpretar padrões temporais

**Habilidades:**
- Criar e manipular DatetimeIndex
- Fazer resample (mudar frequência temporal)
- Calcular médias móveis (rolling windows)
- Decompor séries em componentes
- Tratar missing values em séries temporais
- Criar visualizações temporais eficazes
- Extrair features temporais

---

## Dataset

**Air Quality Dataset (UCI)**
- Dados de qualidade do ar de uma cidade italiana
- Medições horárias de março 2004 a fevereiro 2005
- Variáveis: CO, NO2, temperatura, umidade, etc.
- ~9000 observações

---

## Setup: Imports e Configurações

In [ ]:
# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import zipfile
import io
import requests
import warnings
warnings.filterwarnings('ignore')

# Configurações do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11
sns.set_palette("husl")

print("Setup completo!")

## Parte 1: Carregar e Explorar os Dados

### 1.1 Carregar Air Quality Dataset

In [ ]:
# Carregar dados do UCI Repository
zip_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
r = requests.get(zip_url)
z = zipfile.ZipFile(io.BytesIO(r.content))

# Carregar CSV
with z.open('AirQualityUCI.csv') as f:
    df = pd.read_csv(f, sep=';', decimal=',', na_values=-200)

print("Dados carregados!")
print(f"Shape: {df.shape}")

### 1.2 Primeira Exploração

In [ ]:
# Visualizar primeiras linhas
print("=== PRIMEIRAS LINHAS ===")
display(df.head())

# Informações sobre o dataset
print("\n=== INFO ===")
df.info()

In [ ]:
# Estatísticas descritivas
print("=== ESTATÍSTICAS DESCRITIVAS ===")
display(df.describe())

### 1.3 Combinar Date e Time em Timestamp

O dataset tem duas colunas separadas: 'Date' e 'Time'. Precisamos combiná-las em uma única coluna datetime.

In [ ]:
# Combinar Date e Time em timestamp
df['timestamp'] = pd.to_datetime(
    df['Date'] + ' ' + df['Time'],
    format='%d/%m/%Y %H.%M.%S',
    dayfirst=True,
    errors='coerce'
)

print("=== COLUNA TIMESTAMP CRIADA ===")
print(f"Tipo: {df['timestamp'].dtype}")
print(f"\nPrimeiros valores:")
print(df['timestamp'].head())

### 1.4 Verificar Range Temporal

In [ ]:
print("=== RANGE TEMPORAL ===")
print(f"Início: {df['timestamp'].min()}")
print(f"Fim: {df['timestamp'].max()}")
print(f"Duração: {df['timestamp'].max() - df['timestamp'].min()}")

# Verificar frequência
time_diff = df['timestamp'].diff().mode()[0]
print(f"\nIntervalo mais comum: {time_diff}")

### 1.5 Valores Faltantes

In [ ]:
print("=== VALORES FALTANTES ===")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing': missing,
    'Percent': missing_pct
}).sort_values('Percent', ascending=False)

print(missing_df[missing_df['Missing'] > 0])

---

## Parte 2: Bloco 1 - DatetimeIndex e Manipulação Básica

### 2.1 Definir DatetimeIndex

In [ ]:
# Transformar timestamp em índice
df = df.set_index('timestamp')

print("=== DATETIMEINDEX DEFINIDO ===")
print(f"Tipo do índice: {type(df.index)}")
print(f"\nPrimeiras entradas do índice:")
print(df.index[:5])

### 2.2 Verificar e Ordenar

In [ ]:
# Verificar se está ordenado
print("=== VERIFICAÇÃO DE ORDENAÇÃO ===")
print(f"Está ordenado? {df.index.is_monotonic_increasing}")

# Ordenar se necessário
df = df.sort_index()
print(f"Após sort_index: {df.index.is_monotonic_increasing}")

# Verificar duplicatas
print(f"\nDuplicatas no índice: {df.index.duplicated().sum()}")

# Remover duplicatas se existirem
df = df[~df.index.duplicated(keep='first')]
print(f"Shape após remover duplicatas: {df.shape}")

### 2.3 Extrair Componentes Temporais

In [ ]:
# Criar colunas com componentes temporais
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['hour'] = df.index.hour
df['dayofweek'] = df.index.dayofweek  # 0=Segunda, 6=Domingo
df['quarter'] = df.index.quarter
df['weekday_name'] = df.index.day_name()

print("=== COMPONENTES TEMPORAIS EXTRAÍDOS ===")
print(df[['year', 'month', 'day', 'hour', 'dayofweek', 'weekday_name']].head(10))

In [ ]:
# Valores únicos de cada componente
print("=== VALORES ÚNICOS ===")
print(f"Anos: {df['year'].unique()}")
print(f"Meses: {df['month'].unique()}")
print(f"Dias da semana: {df['weekday_name'].unique()}")

### 2.4 Slicing por Datas

In [ ]:
# IMPORTANTE: Após limpar duplicatas, é mais seguro usar comparações explícitas
# ao invés de slicing com strings parciais

# Um dia específico
march_15 = df[df.index.date == pd.Timestamp('2004-03-15').date()]
print(f"=== DADOS DE 15/03/2004 ===")
print(f"Número de registros: {len(march_15)}")
print(march_15[['CO(GT)', 'T', 'RH']].head())

In [ ]:
# Um mês inteiro
march_2004 = df[(df.index.year == 2004) & (df.index.month == 3)]
print(f"\n=== MARÇO 2004 ===")
print(f"Número de registros: {len(march_2004)}")

# Um ano inteiro
year_2004 = df[df.index.year == 2004]
print(f"\n=== ANO 2004 ===")
print(f"Número de registros: {len(year_2004)}")

In [ ]:
# Range de datas usando comparação de timestamps
spring_2004 = df[(df.index >= '2004-03-01') & (df.index <= '2004-05-31')]
print(f"\n=== PRIMAVERA 2004 (MAR-MAI) ===")
print(f"Número de registros: {len(spring_2004)}")

# Até certa data
early_2004 = df[df.index <= '2004-06-30']
print(f"\n=== ATÉ JUNHO/2004 ===")
print(f"Número de registros: {len(early_2004)}")

### 2.5 Filtrar por Componentes Temporais

In [ ]:
# Filtrar por hora do dia
morning_10am = df[df.index.hour == 10]
print(f"=== TODAS AS 10H DA MANHÃ ===")
print(f"Número de registros: {len(morning_10am)}")

# Fins de semana
weekends = df[df.index.dayofweek >= 5]
print(f"\n=== FINS DE SEMANA ===")
print(f"Número de registros: {len(weekends)}")

# Dias úteis
weekdays = df[df.index.dayofweek < 5]
print(f"\n=== DIAS ÚTEIS ===")
print(f"Número de registros: {len(weekdays)}")

In [ ]:
# Múltiplas condições: dias úteis + horário comercial
business_hours = df[
    (df.index.dayofweek < 5) &  # Segunda a sexta
    (df.index.hour >= 9) &       # Depois das 9h
    (df.index.hour < 18)         # Antes das 18h
]

print(f"=== HORÁRIO COMERCIAL ===")
print(f"Número de registros: {len(business_hours)}")
print(f"Percentual do total: {len(business_hours)/len(df)*100:.1f}%")

### 2.6 Estatísticas por Período

In [ ]:
# Média de CO por hora do dia
hourly_stats = df.groupby(df.index.hour)['CO(GT)'].agg(['mean', 'std', 'min', 'max', 'count'])

print("=== ESTATÍSTICAS DE CO POR HORA DO DIA ===")
display(hourly_stats)

In [ ]:
# Média por dia da semana
weekday_stats = df.groupby(df.index.dayofweek)['CO(GT)'].mean()
weekday_stats.index = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex', 'Sab', 'Dom']

print("\n=== MÉDIA DE CO POR DIA DA SEMANA ===")
print(weekday_stats)

In [ ]:
# Comparar dias úteis vs fins de semana
weekday_co = df[df.index.dayofweek < 5]['CO(GT)'].mean()
weekend_co = df[df.index.dayofweek >= 5]['CO(GT)'].mean()

print("=== DIAS ÚTEIS VS FINS DE SEMANA ===")
print(f"Dias úteis: {weekday_co:.2f} mg/m³")
print(f"Fins de semana: {weekend_co:.2f} mg/m³")
print(f"Diferença: {weekday_co - weekend_co:.2f} mg/m³")
print(f"Variação: {((weekday_co/weekend_co - 1) * 100):.1f}%")

### 2.7 Criar Features Temporais

In [ ]:
# Parte do dia
def get_part_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

df['part_of_day'] = df.index.hour.map(get_part_of_day)

# Fim de semana (binário)
df['is_weekend'] = (df.index.dayofweek >= 5).astype(int)

# Horário de pico
df['is_rush_hour'] = df.index.hour.isin([7,8,9,17,18,19]).astype(int)

print("=== FEATURES TEMPORAIS CRIADAS ===")
print(df[['part_of_day', 'is_weekend', 'is_rush_hour']].head(20))

### 2.8 Features Cíclicas

In [ ]:
# Representação cíclica de componentes temporais

# Hora do dia (0-23)
df['hour_sin'] = np.sin(2 * np.pi * df.index.hour / 24)
df['hour_cos'] = np.cos(2 * np.pi * df.index.hour / 24)

# Dia da semana (0-6)
df['dow_sin'] = np.sin(2 * np.pi * df.index.dayofweek / 7)
df['dow_cos'] = np.cos(2 * np.pi * df.index.dayofweek / 7)

# Mês do ano (1-12)
df['month_sin'] = np.sin(2 * np.pi * df.index.month / 12)
df['month_cos'] = np.cos(2 * np.pi * df.index.month / 12)

print("=== FEATURES CÍCLICAS ===")
print(df[['hour_sin', 'hour_cos', 'dow_sin', 'dow_cos']].head(10))

### 2.9 Encontrar Valores Extremos

In [ ]:
# Pior qualidade do ar (maior CO)
max_co_idx = df['CO(GT)'].idxmax()
max_co_val = df['CO(GT)'].max()

print("=== PIOR QUALIDADE DO AR (CO) ===")
print(f"Quando: {max_co_idx}")
print(f"Valor: {max_co_val:.2f} mg/m³")
print(f"Hora: {max_co_idx.hour}h")
print(f"Dia da semana: {max_co_idx.day_name()}")

# Melhor qualidade do ar
min_co_idx = df['CO(GT)'].idxmin()
min_co_val = df['CO(GT)'].min()

print("\n=== MELHOR QUALIDADE DO AR (CO) ===")
print(f"Quando: {min_co_idx}")
print(f"Valor: {min_co_val:.2f} mg/m³")

In [ ]:
# Top 10 piores momentos
worst_10 = df.nlargest(10, 'CO(GT)')

print("=== TOP 10 PIORES MOMENTOS ===")
print(worst_10[['CO(GT)', 'T', 'RH', 'hour', 'weekday_name']])

---

## Parte 3: Bloco 2 - Resample, Rolling Windows e Agregações

### 3.1 Resample: Downsampling

In [ ]:
# Resample de horário para diário (média)
daily_avg = df['CO(GT)'].resample('D').mean()

print("=== RESAMPLE DIÁRIO (MÉDIA) ===")
print(f"Original: {len(df)} registros (horários)")
print(f"Resampled: {len(daily_avg)} registros (diários)")
print(f"\nPrimeiros valores:")
print(daily_avg.head())

In [ ]:
# Múltiplas agregações
daily_stats = df['CO(GT)'].resample('D').agg(['mean', 'std', 'min', 'max', 'count'])

print("=== ESTATÍSTICAS DIÁRIAS DE CO ===")
display(daily_stats.head(10))

In [ ]:
# Múltiplas colunas, diferentes funções
daily_multi = df.resample('D').agg({
    'CO(GT)': ['mean', 'max'],
    'NO2(GT)': ['mean', 'max'],
    'T': ['mean', 'min', 'max'],
    'RH': 'mean'
})

print("\n=== MÚLTIPLAS COLUNAS E AGREGAÇÕES ===")
display(daily_multi.head())

In [ ]:
# Diferentes períodos
weekly = df['CO(GT)'].resample('W').mean()
monthly = df['CO(GT)'].resample('M').mean()
six_hourly = df['CO(GT)'].resample('6H').mean()

print("=== DIFERENTES FREQUÊNCIAS ===")
print(f"Diário: {len(daily_avg)} registros")
print(f"Semanal: {len(weekly)} registros")
print(f"Mensal: {len(monthly)} registros")
print(f"6 em 6 horas: {len(six_hourly)} registros")

### 3.2 Visualizar Resample

In [ ]:
# Comparar diferentes frequências
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Horário (primeiros 7 dias)
df['CO(GT)'][:168].plot(ax=axes[0], title='Horário (primeiros 7 dias)', alpha=0.7)
axes[0].set_ylabel('CO (mg/m³)')

# Diário
daily_avg.plot(ax=axes[1], title='Diário', color='red')
axes[1].set_ylabel('CO (mg/m³)')

# Semanal
weekly.plot(ax=axes[2], title='Semanal', color='green', linewidth=2)
axes[2].set_ylabel('CO (mg/m³)')

# Mensal
monthly.plot(ax=axes[3], title='Mensal', color='purple', linewidth=2.5)
axes[3].set_ylabel('CO (mg/m³)')

plt.tight_layout()
plt.show()

### 3.3 Rolling Windows: Média Móvel

In [ ]:
# Média móvel de 24 horas
df['CO_MA24'] = df['CO(GT)'].rolling(window=24, min_periods=1).mean()

# Média móvel de 7 dias (168 horas)
df['CO_MA168'] = df['CO(GT)'].rolling(window=168, min_periods=1).mean()

print("=== MÉDIAS MÓVEIS CRIADAS ===")
print(df[['CO(GT)', 'CO_MA24', 'CO_MA168']].head(30))

In [ ]:
# Visualizar médias móveis
plt.figure(figsize=(14, 6))
plt.plot(df.index[:500], df['CO(GT)'][:500], alpha=0.3, label='Original', linewidth=1)
plt.plot(df.index[:500], df['CO_MA24'][:500], label='MA 24h', linewidth=1.5)
plt.plot(df.index[:500], df['CO_MA168'][:500], label='MA 7d (168h)', linewidth=2)
plt.legend()
plt.title('Médias Móveis de CO')
plt.ylabel('CO (mg/m³)')
plt.xlabel('Data')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 3.4 Rolling: Outras Estatísticas

In [ ]:
# Desvio padrão móvel (volatilidade)
df['CO_std24'] = df['CO(GT)'].rolling(window=24, min_periods=1).std()

# Máximo móvel
df['CO_max24'] = df['CO(GT)'].rolling(window=24, min_periods=1).max()

# Mínimo móvel
df['CO_min24'] = df['CO(GT)'].rolling(window=24, min_periods=1).min()

# Mediana móvel
df['CO_median24'] = df['CO(GT)'].rolling(window=24, min_periods=1).median()

print("=== ROLLING STATISTICS ===")
print(df[['CO(GT)', 'CO_MA24', 'CO_std24', 'CO_max24', 'CO_min24']].tail(10))

### 3.5 Comparar Diferentes Janelas

In [ ]:
# Múltiplas janelas
df['CO_MA12'] = df['CO(GT)'].rolling(12, min_periods=1).mean()  # 12h
df['CO_MA24'] = df['CO(GT)'].rolling(24, min_periods=1).mean()  # 24h
df['CO_MA168'] = df['CO(GT)'].rolling(168, min_periods=1).mean()  # 7 dias
df['CO_MA720'] = df['CO(GT)'].rolling(720, min_periods=1).mean()  # 30 dias

# Visualizar
plt.figure(figsize=(14, 6))
df['CO(GT)'].plot(alpha=0.2, label='Original', color='gray', linewidth=0.5)
df['CO_MA12'].plot(label='MA 12h', linewidth=1)
df['CO_MA24'].plot(label='MA 24h', linewidth=1.5)
df['CO_MA168'].plot(label='MA 7d', linewidth=2)
df['CO_MA720'].plot(label='MA 30d', linewidth=2.5)
plt.legend()
plt.title('Múltiplas Médias Móveis')
plt.ylabel('CO (mg/m³)')
plt.xlabel('Data')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 3.6 Shift: Defasagens (Lags)

In [ ]:
# Criar lags
df['CO_lag1'] = df['CO(GT)'].shift(1)  # 1 hora atrás
df['CO_lag24'] = df['CO(GT)'].shift(24)  # 1 dia atrás
df['CO_lag168'] = df['CO(GT)'].shift(168)  # 1 semana atrás

print("=== LAGS CRIADOS ===")
print(df[['CO(GT)', 'CO_lag1', 'CO_lag24', 'CO_lag168']].head(30))

### 3.7 Diff: Diferenças Temporais

In [ ]:
# Diferenças
df['CO_diff1'] = df['CO(GT)'].diff(1)  # Mudança horária
df['CO_diff24'] = df['CO(GT)'].diff(24)  # Mudança dia-a-dia

print("=== DIFERENÇAS ===")
print(df[['CO(GT)', 'CO_diff1', 'CO_diff24']].head(30))

### 3.8 pct_change: Variações Percentuais

In [ ]:
# Variação percentual
df['CO_pct_change1'] = df['CO(GT)'].pct_change(1)
df['CO_pct_change24'] = df['CO(GT)'].pct_change(24)

print("=== VARIAÇÕES PERCENTUAIS ===")
print(df[['CO(GT)', 'CO_pct_change1', 'CO_pct_change24']].head(30))

print(f"\nVariação percentual média dia-a-dia: {df['CO_pct_change24'].mean()*100:.2f}%")

### 3.9 Autocorrelação

In [ ]:
# Calcular autocorrelações para diferentes lags
lags_to_test = [1, 6, 12, 24, 48, 72, 168]
correlations = []

print("=== AUTOCORRELAÇÕES ===")
for lag in lags_to_test:
    corr = df['CO(GT)'].corr(df['CO(GT)'].shift(lag))
    correlations.append((lag, corr))
    print(f"Lag {lag:3d}h: correlação = {corr:.3f}")

In [ ]:
# Plot de autocorrelação
from pandas.plotting import autocorrelation_plot

plt.figure(figsize=(14, 6))
autocorrelation_plot(df['CO(GT)'].dropna())
plt.title('Autocorrelação de CO', fontsize=14)
plt.xlabel('Lag (horas)')
plt.ylabel('Autocorrelação')
plt.axhline(y=0, color='k', linestyle='--')
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.3)
plt.axhline(y=-0.5, color='r', linestyle='--', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 3.10 Detecção de Anomalias com Rolling

In [ ]:
# Detectar anomalias usando z-score móvel
df['CO_rolling_mean'] = df['CO(GT)'].rolling(24, min_periods=1).mean()
df['CO_rolling_std'] = df['CO(GT)'].rolling(24, min_periods=1).std()

# Z-score móvel
df['CO_zscore'] = (df['CO(GT)'] - df['CO_rolling_mean']) / df['CO_rolling_std']

# Anomalias: |z-score| > 3
df['is_anomaly'] = (df['CO_zscore'].abs() > 3).astype(int)

print("=== DETECÇÃO DE ANOMALIAS ===")
anomalies = df[df['is_anomaly'] == 1]
print(f"Total de anomalias: {len(anomalies)}")
print(f"Percentual: {(len(anomalies)/len(df))*100:.2f}%")

print("\nTop 10 anomalias:")
print(anomalies.nlargest(10, 'CO_zscore')[['CO(GT)', 'CO_rolling_mean', 'CO_zscore']])

---

## Parte 4: Bloco 3 - Decomposição de Séries Temporais

### 4.1 Preparar Dados para Decomposição

In [ ]:
# Resample para diário (decomposição funciona melhor com frequência regular)
daily = df['CO(GT)'].resample('D').mean().dropna()

print("=== DADOS PARA DECOMPOSIÇÃO ===")
print(f"Frequência: Diária")
print(f"Registros: {len(daily)}")
print(f"Período: {daily.index.min()} a {daily.index.max()}")
print(f"Missing: {daily.isnull().sum()}")

### 4.2 Decomposição Aditiva

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Decomposição aditiva com sazonalidade semanal
decomposition = seasonal_decompose(
    daily,
    model='additive',
    period=7,  # Sazonalidade semanal
    extrapolate_trend='freq'
)

# Extrair componentes
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

print("=== DECOMPOSIÇÃO COMPLETA ===")
print(f"Trend: {trend.shape}")
print(f"Seasonal: {seasonal.shape}")
print(f"Residual: {residual.shape}")

### 4.3 Visualizar Decomposição

In [ ]:
# Plot de decomposição
fig = decomposition.plot()
fig.set_size_inches(14, 10)
plt.tight_layout()
plt.show()

### 4.4 Analisar Tendência

In [ ]:
print("=== ANÁLISE DE TENDÊNCIA ===")

# Direção geral
start_value = trend.dropna().iloc[0]
end_value = trend.dropna().iloc[-1]
change = end_value - start_value
pct_change = (change / start_value) * 100

print(f"Valor inicial: {start_value:.2f}")
print(f"Valor final: {end_value:.2f}")
print(f"Mudança: {change:.2f} ({pct_change:+.1f}%)")

if pct_change > 5:
    print("Tendência de CRESCIMENTO")
elif pct_change < -5:
    print("Tendência de DECLÍNIO")
else:
    print("Tendência ESTÁVEL")

# Taxa de crescimento média
growth_rate = trend.pct_change().mean() * 100
print(f"\nTaxa de crescimento diária média: {growth_rate:.3f}%")

### 4.5 Analisar Sazonalidade

In [ ]:
print("=== ANÁLISE DE SAZONALIDADE ===")

# Amplitude sazonal
seasonal_amplitude = seasonal.max() - seasonal.min()
print(f"Amplitude sazonal: {seasonal_amplitude:.2f}")

# Força da sazonalidade
total_variation = daily.std()
seasonal_strength = (seasonal.std() / total_variation) * 100
print(f"Força da sazonalidade: {seasonal_strength:.1f}% da variação total")

# Padrão semanal
seasonal_pattern = seasonal.groupby(seasonal.index.dayofweek).mean()
seasonal_pattern.index = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex', 'Sab', 'Dom']
print("\nPadrão semanal (média):")
print(seasonal_pattern)

print(f"\nPior dia: {seasonal_pattern.idxmax()}")
print(f"Melhor dia: {seasonal_pattern.idxmin()}")

### 4.6 Analisar Resíduo

In [ ]:
print("=== ANÁLISE DE RESÍDUO ===")

print(f"Média: {residual.mean():.6f}")  # Deve ser ~0
print(f"Desvio padrão: {residual.std():.3f}")
print(f"Min: {residual.min():.3f}")
print(f"Max: {residual.max():.3f}")

# Outliers no resíduo
residual_threshold = 3 * residual.std()
outliers = residual[residual.abs() > residual_threshold]
print(f"\nOutliers (|residual| > 3σ): {len(outliers)}")

if len(outliers) > 0:
    print("\nMaiores resíduos:")
    top_outliers = residual.abs().nlargest(5)
    for date, value in top_outliers.items():
        print(f"  {date.date()}: {residual[date]:.3f}")

# Autocorrelação do resíduo
residual_autocorr = residual.autocorr(lag=1)
print(f"\nAutocorrelação lag-1: {residual_autocorr:.3f}")
print("(Deve ser próximo de 0 se resíduo é aleatório)")

### 4.7 Comparar Modelos Aditivo vs Multiplicativo

In [ ]:
# Decomposição multiplicativa
decomp_mult = seasonal_decompose(
    daily,
    model='multiplicative',
    period=7,
    extrapolate_trend='freq'
)

# Comparar resíduos
residual_add = decomposition.resid
residual_mult = decomp_mult.resid

print("=== COMPARAÇÃO DE MODELOS ===")
print(f"Aditivo - Resíduo std: {residual_add.std():.3f}")
print(f"Multiplicativo - Resíduo std: {residual_mult.std():.3f}")

if residual_add.std() < residual_mult.std():
    print("\nModelo ADITIVO se ajusta melhor")
else:
    print("\nModelo MULTIPLICATIVO se ajusta melhor")

### 4.8 Série Dessazonalizada

In [ ]:
# Remover sazonalidade
seasonally_adjusted = daily - seasonal

# Visualizar
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Original
daily.plot(ax=axes[0], title='Original', color='blue')
axes[0].set_ylabel('CO (mg/m³)')

# Sazonalidade
seasonal.plot(ax=axes[1], title='Componente Sazonal', color='green')
axes[1].set_ylabel('Sazonal')

# Ajustada (sem sazonalidade)
seasonally_adjusted.plot(ax=axes[2], title='Série Dessazonalizada', color='red')
axes[2].set_ylabel('CO (mg/m³)')

plt.tight_layout()
plt.show()

---

## Parte 5: Bloco 4 - Visualizações e Boas Práticas

### 5.1 Heatmap Temporal

In [ ]:
# Heatmap: dia x hora
pivot = df.pivot_table(
    values='CO(GT)',
    index=df.index.date,
    columns=df.index.hour,
    aggfunc='mean'
)

# Plot primeiro mês
plt.figure(figsize=(14, 10))
sns.heatmap(pivot[:30],
            cmap='RdYlGn_r',
            cbar_kws={'label': 'CO (mg/m³)'},
            linewidths=0.5,
            linecolor='gray')
plt.title('Níveis de CO: Dia x Hora (Primeiro Mês)', fontsize=14)
plt.xlabel('Hora do Dia')
plt.ylabel('Data')
plt.tight_layout()
plt.show()

### 5.2 Box Plot por Período

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Box plot por mês
df.boxplot(column='CO(GT)', by='month', ax=axes[0])
axes[0].set_xlabel('Mês')
axes[0].set_ylabel('CO (mg/m³)')
axes[0].set_title('Distribuição de CO por Mês')
axes[0].get_figure().suptitle('')

# Box plot por dia da semana
df.boxplot(column='CO(GT)', by='dayofweek', ax=axes[1])
axes[1].set_xlabel('Dia da Semana (0=Seg, 6=Dom)')
axes[1].set_ylabel('CO (mg/m³)')
axes[1].set_title('Distribuição de CO por Dia da Semana')
axes[1].get_figure().suptitle('')

plt.tight_layout()
plt.show()

### 5.3 Lag Scatter Plots

In [ ]:
from pandas.plotting import lag_plot

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Lag 1 (1 hora)
lag_plot(df['CO(GT)'], lag=1, ax=axes[0])
axes[0].set_title('Lag 1 (1 hora)')

# Lag 24 (1 dia)
lag_plot(df['CO(GT)'], lag=24, ax=axes[1])
axes[1].set_title('Lag 24 (1 dia)')

# Lag 168 (1 semana)
lag_plot(df['CO(GT)'], lag=168, ax=axes[2])
axes[2].set_title('Lag 168 (1 semana)')

plt.tight_layout()
plt.show()

### 5.4 Comparação de Múltiplas Séries

In [ ]:
# Comparar CO, NO2 e Temperatura
daily_multi = df[['CO(GT)', 'NO2(GT)', 'T']].resample('D').mean()

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# CO
axes[0].plot(daily_multi.index, daily_multi['CO(GT)'], color='red', linewidth=1)
axes[0].set_ylabel('CO (mg/m³)')
axes[0].set_title('Monóxido de Carbono')
axes[0].grid(True, alpha=0.3)

# NO2
axes[1].plot(daily_multi.index, daily_multi['NO2(GT)'], color='blue', linewidth=1)
axes[1].set_ylabel('NO2 (µg/m³)')
axes[1].set_title('Dióxido de Nitrogênio')
axes[1].grid(True, alpha=0.3)

# Temperatura
axes[2].plot(daily_multi.index, daily_multi['T'], color='green', linewidth=1)
axes[2].set_ylabel('Temperatura (°C)')
axes[2].set_title('Temperatura')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Conclusão

Nesta aula, aprendemos:

1. **DatetimeIndex**: Como criar e manipular índices temporais
2. **Resample**: Mudar frequência temporal (downsampling e upsampling)
3. **Rolling Windows**: Calcular médias móveis e estatísticas
4. **Lags e Diferenças**: Criar features temporais
5. **Decomposição**: Separar tendência, sazonalidade e resíduo
6. **Visualizações**: Criar gráficos eficazes para séries temporais

### Próximos Passos

Para aprofundar:
- Modelagem preditiva (ARIMA, Prophet)
- Deep Learning para séries (LSTM, GRU)
- Análise multivariada
- Detecção de anomalias avançada

---